# Count the Number of Powerful Integers

You are given three integers start, finish, and limit. You are also given a 0-indexed string s representing a positive integer.

A positive integer x is called powerful if it ends with s (in other words, s is a suffix of x) and each digit in x is at most limit.

Return the total number of powerful integers in the range [start..finish].

A string x is a suffix of a string y if and only if x is a substring of y that starts from some index (including 0) in y and extends to the index y.length - 1. For example, 25 is a suffix of 5125 whereas 512 is not. 

In [ ]:
class Solution:
    def numberOfPowerfulInt(self, start: int, finish: int, limit: int, s: str) -> int:
        # s가 powerful 조건이 되려면 s에 등장하는 모든 자릿수가 limit 이하여야 함
        for ch in s:
            if int(ch) > limit:
                return 0

        L = len(s)
        M = 10 ** L
        suffix = int(s)

        # Case 1: limit == 9
        if limit == 9:
            # x mod M == suffix 인 x를 등차수열로 세기
            # [start, finish] 범위에서 첫 번째 항 f >= start 를 찾는다.
            r = start % M
            if r <= suffix:
                f = start + (suffix - r)
            else:
                f = start + (M - (r - suffix))
            # f가 범위를 벗어나면 조건을 만족하는 x는 없다.
            if f > finish:
                return 0
            # 항의 수: f, f+M, f+2M, ... <= finish
            return 1 + (finish - f) // M

        # Case 2: limit < 9
        # 자리수 DP로 [0, n]까지의 조건을 만족하는 정수의 개수를 센다.
        from functools import lru_cache

        def count_dp(n: int) -> int:
            digits = list(map(int, str(n)))
            L_bound = len(digits)

            @lru_cache(maxsize=None)
            def dp(pos: int, tight: bool, started: bool, digit_count: int, rem: int) -> int:
                """
                pos: 현재 자릿수 index (0~L_bound)
                tight: 지금까지 상한을 정확히 따라왔는지 여부
                started: 이미 숫자 시작 (즉, 0이 아닌 숫자 선택했는지)
                digit_count: 시작 이후 몇 개의 자릿수를 선택했는지
                rem: 마지막 L자리 (mod M) 값 (아직 L자리가 안 채워진 경우 그냥 전체 숫자)
                """
                # 모든 자리를 처리한 경우
                if pos == L_bound:
                    # 숫자 시작 안 했으면 0, 시작했어도 길이가 L 미만이면 s를 suffix로 가지지 못함
                    if not started or digit_count < L:
                        return 0
                    # rem이 실제 마지막 L자리를 표현함: 만약 digit_count > L이면 
                    # 최근 L자리만 유지한 상태이므로, 이미 rem == (x mod M)임.
                    return 1 if rem == suffix else 0

                res = 0
                # 상한값: 현재 자릿수에 넣을 수 있는 최대 숫자
                up = digits[pos] if tight else limit

                # 만약 아직 시작하지 않은 경우에는 0도 선택 가능(계속 leading zero)
                for d in range(0, up + 1):
                    # d가 allowed digit 범위 내에 있는지 검사.
                    if d > limit:
                        continue

                    new_started = started or (d != 0)
                    new_digit_count = digit_count + 1 if new_started else digit_count

                    # 새로운 rem를 업데이트. 만약 아직 시작하지 않았다면 rem은 그대로 0.
                    new_rem = rem
                    if new_started:
                        # 만약 아직 자릿수가 L 미만이면 단순히 rem*10 + d
                        # 만약 이미 L자리 이상이면, 최근 L자리를 갱신한다.
                        new_rem = (rem * 10 + d) % M

                    # new_tight: 이전까지 tight했으며 현재 d가 up에 딱 맞을 때.
                    new_tight = tight and (d == up)

                    res += dp(pos + 1, new_tight, new_started, new_digit_count, new_rem)
                return res

            return dp(0, True, False, 0, 0)

        # count_dp는 [0, n]까지의 개수를 세므로
        ans = count_dp(finish) - count_dp(start - 1)
        return ans